In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
import pandas as pd
import numpy as np

## **1. Cargando los Datasets con los que vamos a trabajar**

In [89]:
dir_datasets = '/content/drive/MyDrive/DataScience/2. Proyecto de Data Science con Python - YT/0.Dataset/{}'

df_data_historica = pd.read_csv(dir_datasets.format('fifa_worldcup_matches.csv'))
df_fixture = pd.read_csv(dir_datasets.format('fifa_worldcup_fixture.csv'))
df_data_faltante = pd.read_csv(dir_datasets.format('fifa_worldcup_missing_data.csv'))

# **2. Limpiando df_fixture**

In [90]:
# vamos a limpiar las columnas home y away de los espacios en blanco
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

# **3. Limpiando df_missing_data y concatenar al dataframe orginal(df_data_historica)**

**3.1 Revisamos si hay data faltante en las columnas y luego eliminamos**

In [91]:
df_data_faltante[df_data_faltante['home'].isnull()]
df_data_faltante.dropna(inplace=True)

**3.2 Ahora vamos a concatenar el dataframe orginal historica con el dataframe faltante**

In [92]:
df_data_historica = pd.concat([df_data_historica, df_data_faltante], ignore_index=True)
df_data_historica

,home,score,away,year
0,France,4–1,Mexico,1930
1,Argentina,1–0,France,1930
2,Chile,3–0,Mexico,1930
3,Chile,1–0,France,1930
4,Argentina,6–3,Mexico,1930
...,...,...,...,...
896,Spain,0–1,Switzerland,2010
897,Chile,1–0,Switzerland,2010
898,Spain,2–0,Honduras,2010
899,Chile,1–2,Spain,2010


**3.3 Ahora vamos a eliminar duplicados que podria tener un dataframe y a ordenarlo**

In [93]:
df_data_historica.drop_duplicates(inplace=True)
df_data_historica.sort_values('year', inplace=True)
df_data_historica

,home,score,away,year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
419,Brazil,2–0,Costa Rica,2018
420,Serbia,1–2,Switzerland,2018
421,Serbia,0–2,Brazil,2018
408,France,1–0,Peru,2018


**3.4 Hay un partido que no se desputo, por lo cual no tiene marcador, entonces hay que eliminarlo**

In [94]:
df_data_historica[df_data_historica['score']=='w/o[a]'] 

,home,score,away,year
37,Sweden,w/o[a],Austria,1938


**3.5 Vamos a eliminar una fila que no tiene marcador, porque uno de los dos equipos no se presento**

In [95]:
index_eliminar = df_data_historica[df_data_historica['score']=='w/o[a]'].index

**3.6 Eliminamos la fila del partido que no se jugo**

In [96]:
df_data_historica.drop(index = index_eliminar, inplace=True)

**3.7 Ahora seguimos buscando en la columna score que no contenga digitos**

In [97]:
df_data_historica[df_data_historica['score'].str.contains('[^\d–]')]

,home,score,away,year
34,Italy,2–1 (a.e.t.),Czechoslovakia,1934
27,Italy,1–1 (a.e.t.),Spain,1934
24,Austria,3–2 (a.e.t.),France,1934
48,Brazil,1–1 (a.e.t.),Czechoslovakia,1938
42,Czechoslovakia,3–0 (a.e.t.),Netherlands,1938
...,...,...,...,...
443,Spain,1–1 (a.e.t.),Russia,2018
444,Croatia,1–1 (a.e.t.),Denmark,2018
448,Colombia,1–1 (a.e.t.),England,2018
452,Russia,2–2 (a.e.t.),Croatia,2018


**3.8 Ahora vamos a eliminar (a.e.t.) que esta dentro de algunas filas, con el metodo replace**

In [98]:
df_data_historica['score'] = df_data_historica['score'].str.replace('[^\d–]', '', regex=True)

**3.9 Verificamos que se reeplacen**

In [102]:
df_data_historica[df_data_historica['score'].str.contains('[^\d–]')]

,home,score,away,year,HomeGoals,AwayGoals


**3.10 Ahora limpiamos las columnas home y away data historica**

In [103]:
df_data_historica['home'] = df_data_historica['home'].str.strip()
df_data_historica['away'] = df_data_historica['away'].str.strip()

**3.11 Ahora vamos a separar el score en dos columnas, vamos a tener los goles en diferentes columnas**

In [104]:
df_data_historica[['HomeGoals', 'AwayGoals']] = df_data_historica['score'].str.split('–', expand=True)

**3.12 Ahora vamos a eliminar la columna score**

In [105]:
df_data_historica.drop('score', axis=1, inplace=True)

In [106]:
df_data_historica

,home,away,year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
17,Uruguay,Argentina,1930,4,2
16,Uruguay,Yugoslavia,1930,6,1
15,Argentina,United States,1930,6,1
14,Paraguay,Belgium,1930,1,0
...,...,...,...,...,...
419,Brazil,Costa Rica,2018,2,0
420,Serbia,Switzerland,2018,1,2
421,Serbia,Brazil,2018,0,2
408,France,Peru,2018,1,0


**3.13 Ahora vamos a renombrar las columnas del DataFrame**

In [108]:
df_data_historica.rename(columns={'home': 'HomeTeam',
                                  'away': 'AwayTeam',
                                  'year': 'Year'}, inplace=True)

In [109]:
df_data_historica

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
17,Uruguay,Argentina,1930,4,2
16,Uruguay,Yugoslavia,1930,6,1
15,Argentina,United States,1930,6,1
14,Paraguay,Belgium,1930,1,0
...,...,...,...,...,...
419,Brazil,Costa Rica,2018,2,0
420,Serbia,Switzerland,2018,1,2
421,Serbia,Brazil,2018,0,2
408,France,Peru,2018,1,0


**3.14 Ahora debemos cambiar el tipo de dato de las columnas HomeGoals y AwayGoals**

In [110]:
df_data_historica.dtypes

HomeTeam     object
AwayTeam     object
Year          int64
HomeGoals    object
AwayGoals    object
dtype: object

In [111]:
df_data_historica = df_data_historica.astype({'HomeGoals': int, 'AwayGoals': int, 'Year': int})

In [112]:
df_data_historica.dtypes

HomeTeam     object
AwayTeam     object
Year          int64
HomeGoals     int64
AwayGoals     int64
dtype: object

**3.15 Ahora vamos agregar una nueva columna para tener la cantidad de goles totales por partido**

In [114]:
df_data_historica['TotalGoals'] = df_data_historica['HomeGoals'] + df_data_historica['AwayGoals']
df_data_historica

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
419,Brazil,Costa Rica,2018,2,0,2
420,Serbia,Switzerland,2018,1,2,3
421,Serbia,Brazil,2018,0,2,2
408,France,Peru,2018,1,0,1


# **4. Exportamos los DataFrames a un CSV con la informacion ya limpia**

In [116]:
# Exportar el DataFrame a un archivo CSV
df_data_historica.to_csv('/content/drive/MyDrive/DataScience/2. Proyecto de Data Science con Python - YT/0.Dataset/cleaned_fifa_worldcup_matches.csv', index=False)
df_fixture.to_csv('/content/drive/MyDrive/DataScience/2. Proyecto de Data Science con Python - YT/0.Dataset/cleaned_fifa_worldcup_fixture.csv', index=False)


**4.1 Verificar los números de partidos**

In [125]:
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

for year in years:
    print(year, ' jugaron:', len(df_data_historica[df_data_historica['Year']==year]))

1930  jugaron 18
1934  jugaron 17
1938  jugaron 18
1950  jugaron 22
1954  jugaron 26
1958  jugaron 35
1962  jugaron 32
1966  jugaron 32
1970  jugaron 32
1974  jugaron 38
1978  jugaron 38
1982  jugaron 52
1986  jugaron 52
1990  jugaron 52
1994  jugaron 52
1998  jugaron 64
2002  jugaron 64
2006  jugaron 64
2010  jugaron 64
2014  jugaron 64
2018  jugaron 64
